In [56]:
import pandas as pd 
import datetime  #to convert the data and time 
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
#Importing Required Libraries

In [68]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs= []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [170107, 170114, 170128, 170204,  170211, 170218, 170225 ,
            170304, 170304, 170311, 170311, 170318, 170325, 170401, 170408, 170415, 170422, 170429]
turnstiles_df = get_data(week_nums)

#the chosen data  In this analysis we use data from the first half of 2017. Data size is over 5 million
#-	Loading the dataset into Python   

In [53]:
turnstiles_df.head()
#in this analysis we use data for the first quarter of 2017 
#the data includes 3537242 rows × 11 columns

STATION  C/A   UNIT  SCP       DATE      
1 AV     H007  R248  00-00-00  01/01/2017    1530
                               01/02/2017    1986
                               01/03/2017    3809
                               01/04/2017    3679
                               01/05/2017    4305
Name: ENTRIES, dtype: int64

In [ ]:
#C/A - Control Area name/Booth name. This is the internal identification of a booth at a given station.
#Unit - Remote unit ID of station.
#SCP - Subunit/Channel/position represents a specific address for a given device. STATION - Name assigned to the subway station by operations planning. This name is used in all references to stations, 
#as well as in debit/credit purchase receipts, and customer’s bank activity statements.

In [ ]:
#date = Represents the date (MM-DD-YY)
#time = Represents the time (hh:mm:ss) for a scheduled audit event
#datetime = date + time (MM-DD-YY hh:mm:ss)
#desc = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
#entries = The comulative entry register value for a device
#exits = The cumulative exit register value for a device

In [36]:
np.array(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES',
 'EXITS'] )

array(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE',
       'TIME', 'DESC', 'ENTRIES', 'EXITS'], dtype='<U8')

In [65]:
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

In [ ]:
#to facilitate the analysis , I integrated the data and time together in the same column .

In [ ]:
turnstiles_df

In [67]:
## gets the number of entries and exits
turnstiles_df =turnstiles_df.groupby(['STATION','C/A','UNIT','SCP','DATE']).ENTRIES.max() - turnstiles_df.groupby(['STATION','C/A','UNIT','SCP','DATE']).ENTRIES.min()
turnstiles_df = turnstiles_df.groupby(['STATION','C/A','UNIT','SCP','DATE']).EXITS.max() - turnstiles_df.groupby(['STATION','C/A','UNIT','SCP','DATE']).EXITS.min()


AttributeError: 'SeriesGroupBy' object has no attribute 'ENTRIES'

In [80]:
#checking the uniqueness 
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
2715881,R408,R449,00-00-00,E 149 ST,2017-03-10 08:00:00,4
2717237,R408,R449,00-00-02,E 149 ST,2017-03-10 08:00:00,4
2133096,R158,R084,00-06-00,59 ST COLUMBUS,2017-03-05 03:00:00,4
2716559,R408,R449,00-00-01,E 149 ST,2017-03-10 08:00:00,4
2790611,R504,R276,00-00-01,VERNON-JACKSON,2017-02-27 15:00:00,4


In [ ]:
#we are going to find the daily entries rather than cumulative 

In [83]:
turnstiles_daily = (turnstiles_df
                        .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False)
                        .ENTRIES.first())

In [84]:
turnstiles_daily

,C/A,UNIT,SCP,STATION,DATE,ENTRIES
0,A002,R051,02-00-00,59 ST,01/01/2017,5992718
1,A002,R051,02-00-00,59 ST,01/02/2017,5993559
2,A002,R051,02-00-00,59 ST,01/03/2017,5994505
3,A002,R051,02-00-00,59 ST,01/04/2017,5996263
4,A002,R051,02-00-00,59 ST,01/05/2017,5997901
...,...,...,...,...,...,...
522003,TRAM2,R469,00-05-01,RIT-ROOSEVELT,04/25/2017,5554
522004,TRAM2,R469,00-05-01,RIT-ROOSEVELT,04/26/2017,5554
522005,TRAM2,R469,00-05-01,RIT-ROOSEVELT,04/27/2017,5554
522006,TRAM2,R469,00-05-01,RIT-ROOSEVELT,04/28/2017,5554


In [91]:
turnstiles_daily[['PRIVE_DATE','PRIVE_ENTRY']]=(turnstiles_daily
                                                .groupby(['C/A','UNIT','SCP','STATION'])['DATE','ENTRIES']
                                                .apply(lambda grp: grp.shift(1)))

<ipython-input-91-f8d086f6a089>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  turnstiles_daily[['PRIVE_DATE','PRIVE_ENTRY']]=(turnstiles_daily


In [89]:
turnstiles_daily.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'DATE', 'ENTRIES', 'PRIVE_DATE',
       'PRIVE_ENTRY'],
      dtype='object')

In [90]:
turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PRIVE_ENTRY"]].head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PRIVE_DATE,PRIVE_ENTRY
111,A002,R051,02-00-00,59 ST,12/31/2016,5991546,04/28/2017,6156153.0
223,A002,R051,02-00-01,59 ST,12/31/2016,5475418,04/28/2017,5579067.0
335,A002,R051,02-03-00,59 ST,12/31/2016,841884,04/28/2017,903880.0
446,A002,R051,02-03-01,59 ST,12/31/2016,122389,04/28/2017,262644.0
558,A002,R051,02-03-02,59 ST,12/31/2016,5209470,04/28/2017,5349541.0


In [94]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PRIVE_ENTRY"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["PRIVE_ENTRY"])
        counter = min(row["ENTRIES"], row["PRIVE_ENTRY"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)


1241453459 11978209.0
11885857 1241122778.0
2173 10201560.0
10122631 73317.0
67108981 135340.0
128516 67245263.0
100664284 3701657.0
1429 100727135.0
3697537 3181.0
67108864 832.0
100663315 67108865.0
832 100663332.0
1745 1431254.0
1424402 104140.0
2843 6926368.0
6829565 13554.0
419299435 285113520.0
285016014 419273590.0
153 284994405.0
284946825 56085.0
30 8186672.0
8185618 106039.0
2031090592 10515250.0
10325789 2031073365.0
117484830 471282.0
393089 117492085.0
458752 4773394.0
4695802 475802.0
1946191240 7661697.0
7605883 1946198651.0
838882067 3322993.0
3211172 838922600.0
1039 7009645.0
6840587 113974.0
2007 2945113.0
2901408 28743.0
150994944 66.0
63 150994975.0
68 150994944.0
150994944 68.0
1053 3315326.0
3188921 84379.0
262186 6576434.0
6484072 269592.0
2580 9417390.0
9304041 11553.0
67109041 546902.0
117440541 67109932.0
208 117440541.0
2039 4117180.0
4102042 20964.0
2130669175 8476560.0
8444666 2130514147.0
1825 1111672.0
117440740 838749.0
812774 117467272.0
3627 2740774.0

In [95]:
station_daily = turnstiles_daily.groupby(["STATION", "DATE"])[['DAILY_ENTRIES']].sum().reset_index()
station_daily.head()

,STATION,DATE,DAILY_ENTRIES
0,1 AV,01/01/2017,0.0
1,1 AV,01/02/2017,10826.0
2,1 AV,01/03/2017,12141.0
3,1 AV,01/04/2017,21190.0
4,1 AV,01/05/2017,22192.0
